# Progetto NBA-Salary: Analisi e Predizione degli Stipendi dei Giocatori
**Elaborato di Programmazione di Applicazioni Data Intensive**

**Autore:** [Alessio Bifulco]

## 1. Introduzione al Progetto

### 1.1. Contesto e Obiettivo
L'NBA (National Basketball Association) è una delle leghe sportive più ricche e seguite al mondo, con un giro d'affari multimiliardario in cui gli stipendi dei giocatori rappresentano una delle voci di costo più significative per le squadre. Comprendere quali fattori determinano il valore economico di un giocatore è una questione cruciale per la gestione di un team.

Questo progetto si pone tre obiettivi principali:
1.  **Analizzare l'evoluzione degli stipendi** nell'NBA nel decennio 2010-2020, con un focus specifico su come la distribuzione della ricchezza sia cambiata tra i diversi ruoli (playmaker, ali, centri).
2.  **Sviluppare un modello di machine learning** in grado di predire lo stipendio di un giocatore basandosi sulle sue statistiche di performance della stagione precedente.
3.  **Utilizzare il modello finale per una valutazione oggettiva**, confrontando lo stipendio predetto ("valore di mercato stimato") con lo stipendio reale di un giocatore, al fine di identificare atleti potenzialmente sottopagati o strapagati.

### 1.2. I Dataset
Per raggiungere i nostri obiettivi, utilizzeremo due dataset distinti:
- **Dataset Contratti (`contracts_df`):** Una raccolta di contratti firmati da giocatori NBA tra il 2010 e il 2020. La sua caratteristica fondamentale è che associa a ogni contratto le statistiche complete del giocatore nella stagione *precedente* alla firma. Questo lo rende ideale per il nostro obiettivo di predizione.
- **Dataset Statistiche Generali (`general_stats_df`):** Un dataset più ampio contenente le statistiche aggregate di tutti i giocatori NBA dal 1950. Questo dataset sarà fondamentale nella fase di analisi esplorativa per fornire un contesto sull'evoluzione dello stile di gioco nella lega.

### 1.3. Librerie e Impostazioni Iniziali
Iniziamo importando le librerie Python fondamentali per la manipolazione dei dati, l'analisi numerica e la visualizzazione.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Librerie importate e impostazioni pronte.")

Librerie importate e impostazioni pronte.


---

## 2. Caricamento e Ispezione Iniziale dei Dati

Il primo passo operativo è caricare i nostri dataset e condurre un'ispezione preliminare per comprenderne la struttura, i tipi di dato e la presenza di eventuali problemi come valori mancanti.

### 2.1. Caricamento dei Dati Locali

I dataset sono stati scaricati e organizzati in una struttura di cartelle locale. Ora li carichiamo in pandas specificando i percorsi relativi alla posizione del nostro notebook.

In [ ]:
import os
import pandas as pd


path_base = '../dataset/'

contratti_folder_name = 'salaryDataset'


nome_file_contratti = 'nba_contracts_history.csv'


percorso_completo_contratti = os.path.join(path_base, contratti_folder_name, nome_file_contratti)


try:
    contracts_df = pd.read_csv(percorso_completo_contratti)
    
    print("✅ Dataset caricati con successo dai percorsi locali!")
    
except FileNotFoundError as e:
    print(f"❌ ERRORE: File non trovato.")
    print(f"Dettaglio errore: {e}")
    print("\nControlla che il nome del file delle statistiche ('Seasons_Stats.csv') sia corretto.")

if 'contracts_df' in locals():
    print("\nPrime 5 righe del DataFrame dei Contratti:")
    display(contracts_df.head())


✅ Dataset caricati con successo dai percorsi locali!

Prime 5 righe del DataFrame dei Contratti:


,NAME,CONTRACT_START,CONTRACT_END,AVG_SALARY,AGE,GP,W,L,MIN,PTS,...,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,+/-
0,Wesley Matthews,2019,2020,2564753.0,32.0,69.0,27.0,42.0,2091.0,840.0,...,81.0,32.0,138.0,170.0,160.0,91.0,54.0,17.0,160.0,-90.0
1,Brook Lopez,2015,2017,21165675.0,27.0,72.0,34.0,38.0,2100.0,1236.0,...,81.4,214.0,321.0,535.0,50.0,104.0,43.0,126.0,206.0,-104.0
2,DeAndre Jordan,2011,2014,10759763.5,22.0,80.0,31.0,49.0,2047.0,566.0,...,45.2,207.0,368.0,575.0,42.0,101.0,41.0,142.0,255.0,-100.0
3,Markieff Morris,2015,2018,8143323.5,25.0,82.0,39.0,43.0,2581.0,1258.0,...,76.3,110.0,396.0,506.0,192.0,170.0,101.0,39.0,245.0,105.0
4,Dwight Howard,2018,2019,13410739.0,32.0,81.0,35.0,46.0,2463.0,1347.0,...,57.4,255.0,757.0,1012.0,105.0,208.0,48.0,131.0,249.0,89.0
